In [1]:
!pip install selenium

In [2]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By
import time

from selenium.common.exceptions import NoSuchElementException, TimeoutException

import urllib.request

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
import re

from selenium.webdriver.common.keys import Keys

In [129]:
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()

Write a python program which searches all the product under a particular product from www.amazon.in. The 
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for 
guitars

In [4]:
driver.get("https://www.amazon.in/")

In [5]:
#If user input is ‘guitar’. Then search for guitars
inputField = input("Enter the Product Name: ")

Enter the Product Name: Guitar


In [6]:
searchProduct=driver.find_element(By.CLASS_NAME,"nav-input")
searchProduct.send_keys(inputField+'s')

enterSearch=driver.find_element(By.ID,"nav-search-submit-button")
enterSearch.click()

In the above question, now scrape the following details of each product listed in first 3 pages of your search 
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then 
scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [7]:
start=0
end=3 #3 page content

productLink=[]

for page in range(start,end):
    links=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]') 
    for l in links:
        link=l.get_attribute('href')
        productLink.append(link)

    nextPage=driver.find_elements(By.XPATH,'//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[29]/div/div/span') 
    
#len(productLink)
productLink

['https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxNDU4OTU5MzIwNDE1Mzg5OjE2ODcxODg0NzE6c3BfYXRmOjIwMTIxNzg0MjQwOTk4OjowOjo&url=%2FKadence-Frontier-Acoustic-Guitar-Equalizer%2Fdp%2FB01HRIHJWY%2Fref%3Dsr_1_1_sspa%3Fcrid%3D151MU6CVBYL9C%26keywords%3DGuitars%26qid%3D1687188471%26sprefix%3Dguitars%252Caps%252C226%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1',
 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxNDU4OTU5MzIwNDE1Mzg5OjE2ODcxODg0NzE6c3BfYXRmOjIwMTQ2OTY1MzcwODk4OjowOjo&url=%2FPremium-Spruce-Top-Dreadnought-Acoustic-Guitar%2Fdp%2FB0BJK1CWSN%2Fref%3Dsr_1_2_sspa%3Fcrid%3D151MU6CVBYL9C%26keywords%3DGuitars%26qid%3D1687188471%26sprefix%3Dguitars%252Caps%252C226%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1',
 'https://www.amazon.in/sspa/click?ie=UTF8&spc=MToxNDU4OTU5MzIwNDE1Mzg5OjE2ODcxODg0NzE6c3BfYXRmOjIwMDk2NjA4NzUwNDk4OjowOjo&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB01GDZ46AA%2Fref%3Dsr_1_3_sspa%3Fcrid%3D151MU6CVBYL9C%26keywords%3DGui

In [8]:
brandNames=[]
productTitles=[]
prices=[]
returns=[]
deliveries=[]
availables=[]
urls=[]

for i in productLink:
    driver.get(i)
    time.sleep(3)
    
    #scraping guitar brand name
    try:
        brandName=driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]')
        name=brandName.text      
        if name!='':
            brandNames.append(name)
        else:
            brandNames.append("--")      
    except NoSuchElementException:
        brandNames.append('Error in BrandName')
    
    #scraping guitar title
    try:        
        pTitle=driver.find_element(By.XPATH,'//span[@id="productTitle"]')
        title=pTitle.text
        if title!='':
            productTitles.append(title)
        else:
            productTitles.append("--") 
    except NoSuchElementException:
        productTitles.append('Error in ProductTiles')
        
    #scraping guitar price
    try:        
        pPrice=driver.find_element(By.XPATH,'//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[2]')
        price=pPrice.text
        if price!='':
            prices.append(price)
        else:
            prices.append("--")         
    except NoSuchElementException:
        prices.append('Error in Prices')
        
    #scraping guitar returns
    try:        
        pReturn=driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]/span/div[2]')
        returnP=pReturn.text
        if returnP!='':
            returns.append(returnP)
        else:
            returns.append("--")
    except NoSuchElementException:
        returns.append('Error in Returns')
    
    #scraping guitar delivery date
    try:
        pDelivery=driver.find_element(By.XPATH,'//*[@id="mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE"]/span')
        delivery=pDelivery.text
        if delivery!='':
            deliveries.append(delivery)
        else:
            deliveries.append("--")       
    except NoSuchElementException:
        deliveries.append('Error in Deliveries')
        
    #scraping guitar availability
    try:
        pAvailable=driver.find_element(By.XPATH,'//*[@id="availability"]')
        available=pAvailable.text
        if available!='':
            availables.append(available)
        else:
            availables.append("--")                 
    except NoSuchElementException:
        availables.append('Error in Available')

In [9]:
data={'Brand':brandNames, 'Title':productTitles, 'Returns':returns, 'Delivery':deliveries, 'Available':availables, 
      'URL':productLink}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
df

,Brand,Title,Returns,Delivery,Available,URL
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,7 days Replacement,"FREE delivery Friday, 23 June. Details",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,VAULT,Vault DA40 41 Inch Premium Solid Spruce-Top Dr...,7 days Replacement,"FREE delivery Wednesday, 21 June. Order within...",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Kadence,Kadence Frontier guitar with Online Guitar lea...,7 days Replacement,"FREE delivery Wednesday, 21 June. Details",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Cort,Cort AF500C Standard Series Open Pore Cutaway ...,7 days Returnable,"FREE delivery Wednesday, 21 June. Order within...",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,JUAREZ,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",7 days Replacement,"FREE delivery Wednesday, 21 June. Details",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...
253,Ibanez,Ibanez MD39C-NT Rose Wood Acoustic Guitar (Nat...,7 days Replacement,"FREE delivery Wednesday, 21 June. Details",In stock,https://www.amazon.in/Ibanez-IBA-MD39CNT-Acous...
254,JUAREZ,Juârez JRZ38C/WH 6 Strings Acoustic Guitar 38 ...,7 days Replacement,"FREE delivery Wednesday, 21 June. Details",In stock,https://www.amazon.in/Ju%C3%A2rez-JRZ38C-WH-St...
255,YAMAHA,"YAMAHA F310, 6-Strings Rose Wood Acoustic Guit...",7 days Replacement,"FREE delivery Tuesday, 27 June. Details",In stock,https://www.amazon.in/Yamaha-F310-6-Strings-Ac...
256,JUAREZ,"Juarez Acoustic Guitar, 38 Inch Cutaway with P...",7 days Replacement,"FREE delivery Wednesday, 21 June. Details",In stock,https://www.amazon.in/Juarez-Acoustic-Guitar-C...


In [10]:
df.to_csv('product.csv')

In [11]:
#######################images in google#############################

Write a python program to access the search bar and search button on images.google.com and scrape 10 
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [12]:
googleImages=driver.get("https://images.google.co.uk/")

In [13]:
try:
    wait=WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "dbsFrd")))
    x=wait.find_element(By.ID,"L2AGLb")
    x.click()
except TimeoutException as e:
    print(e)    

In [14]:
#https://www.geeksforgeeks.org/download-google-image-using-python-and-selenium/
    
search_item = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

for item in search_item:
    driver.get("https://images.google.co.uk/")
    time.sleep(5)

    # Finding the search box
    box = driver.find_element(By.CLASS_NAME, 'gLFyf') #driver.find_element(By.TAG_NAME, "input") #
    box.send_keys(item)
    
    enterSearch = driver.find_element(By.XPATH, "//button[@class='Tg7LZd']")
    enterSearch.click()

    # Loop to capture and save each image
    for i in range(1, 11):    
        try:

          # XPath of each image
            img = driver.find_element(By.XPATH, 
                '//*[@id="islrg"]/div[1]/div[' +
              str(i) + ']/a[1]/div[1]/img')

            # Enter the location of folder in which
            # the images will be saved
            img.screenshot(r"C:\Users\hp\Desktop\DataScience\Selenium\ErrorExceptionHandling" + 
                           item + " (" + str(i) + ").png")
            print("Downloading ", item, i)
            # Each new screenshot will automatically
            # have its name updated

            # Just to avoid unwanted errors
            time.sleep(0.2)

        except:

            # if we can't find the XPath of an image,
            # we skip to the next image
            continue

In [15]:
############Flipcart - Smartphone##############

Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand 
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV. 

In [16]:
flipcart=driver.get("https://www.flipkart.com/")

In [17]:
try:
    wait=WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "_2hriZF")))
    x=wait.find_element(By.CLASS_NAME,"_2KpZ6l")
    x.click()
except TimeoutException:
    print('no alert')

In [18]:
#search for smartphone
search=driver.find_element(By.CLASS_NAME,"_3704LK")
search.send_keys('smartphone')

enterSearch=driver.find_element(By.CLASS_NAME,"L0Z3Pu")
enterSearch.click()

In [19]:
load=WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='_1AtVbE']")))

In [20]:
phoneUrl=[]
pUrl=driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for p in pUrl:
    link=p.get_attribute('href')
    phoneUrl.append(link)
    
#phoneUrl

In [21]:
sbrandNames=[]
sName=[]
sColour=[]
sRam=[]
sStorage=[]
sCamera1=[]
sCamera2=[]
sSize=[]
sBattery=[]
sPrice=[]

for i in phoneUrl:
    driver.get(i)
    time.sleep(3)
        
    #scraping brand name
    try:
        brandName = driver.find_element(By.XPATH, "//span[@class='B_NuCI']")
        name=brandName.text.split()[0]
        if name!='':
            sbrandNames.append(name)
        else:
            sbrandNames.append("--")    
    except NoSuchElementException:
        sbrandNames.append('Error1')
        
    #scraping name
    try:
        phoneTitle=driver.find_element(By.CLASS_NAME, 'yhB1nd')
        title=phoneTitle.text
        if title!='':
            sName.append(title)
        else:
            sName.append('--')
    except NoSuchElementException:
        sName.append('Error2')
        
    #scraping color
    try:
        phoneColor=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div/div[1]/table/tbody/tr[4]/td[2]')
        color=phoneColor.text
        if color!='':
            sColour.append(color)
        else:
            sColour.append('--')
    except NoSuchElementException:
        sColour.append('Error3')
        
    #scraping Ram
    try:
        phoneRam=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div/div/div[2]/ul/li[1]')
        ram=phoneRam.text[:-3].split('|')[0]
        if ram!='':
            sRam.append(ram)
        else:
            sRam.append('--')
    except NoSuchElementException:
        sRam.append('Error4')
        
    #scraping Rom
    try:
        phoneRom=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div/div/div[2]/ul/li[1]')
        rom=phoneRom.text[:-3].split('|')[1]
        if rom!='':
            sStorage.append(rom)
        else:
            sStorage.append('--')
    except NoSuchElementException:
        sStorage.append('Error5')
            
    #scraping Camera1
    try:
        phoneCamera1=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div/div/div[2]/ul/li[3]')
        camera1=phoneCamera1.text.split('|')[0]
        if camera1!='':
            sCamera1.append(camera1)
        else:
            sCamera1.append('--')
    except NoSuchElementException:
        sCamera1.append('Error6')
        
    #scraping Camera2
    try:
        phoneCamera2=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div/div/div[2]/ul/li[3]')
        camera2=phoneCamera1.text.split('|')[1]
        if camera2!='':
            sCamera2.append(camera2)
        else:
            sCamera2.append('--')
    except NoSuchElementException:
        sCamera2.append('Error7')
        
    #scraping size
    try:
        phoneSize=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div/div/div[2]/ul/li[2]')
        size=phoneSize.text.split('+')[0]
        if size!='':
            sSize.append(size)
        else:
            sSize.append('--')
    except NoSuchElementException:
        sSize.append('Error8')
        
    #scraping Battery
    try:
        phoneBattery=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div/div/div[2]/ul/li[4]')
        battery=phoneBattery.text
        if battery!='':
            sBattery.append(battery)
        else:
            sBattery.append('--')
    except NoSuchElementException:
        sBattery.append('Error9')  
    
    #scraping price
    try:
        phonePrice=driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[4]/div[1]/div/div[1]')
        price=phonePrice.text
        if price!='':
            sPrice.append(price)
        else:
            sPrice.append('--')
    except NoSuchElementException:
        sPrice.append('Error10')

In [22]:
data={'Brand':sbrandNames, 'Title':sName, 'Colour':sColour, 'RAM-Size':sRam, 'Storage':sStorage, 'PrimaryCamera':sCamera1, 
      'SecondaryCamera':sCamera2, 'DisplaySize':sSize, 'Battery':sBattery, 'Price':sPrice, 'URL':phoneUrl}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
df

,Brand,Title,Colour,RAM-Size,Storage,PrimaryCamera,SecondaryCamera,DisplaySize,Battery,Price,URL
0,SAMSUNG,"SAMSUNG Galaxy F13 (Waterfall Blue, 128 GB) (...",Waterfall Blue,4 GB RAM,128 GB ROM,50MP + 5MP + 2MP,8MP Front Camera,16.76 cm (6.6 inch) Full HD,6000 mAh Lithium Ion Battery,"₹10,699",https://www.flipkart.com/samsung-galaxy-f13-wa...
1,Infinix,"Infinix Smart 7 HD (Green Apple, 64 GB) (2 GB...",Green Apple,2 GB RAM,64 GB ROM,8MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,5000 mAh Battery,"₹6,199",https://www.flipkart.com/infinix-smart-7-hd-gr...
2,Infinix,"Infinix SMART 7 (Azure Blue, 64 GB) (4 GB RAM)",Azure Blue,4 GB RAM,64 GB ROM,13MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,6000 mAh Battery,"₹7,299",https://www.flipkart.com/infinix-smart-7-azure...
3,Infinix,"Infinix SMART 7 (Night Black, 64 GB) (4 GB RAM)",Night Black,4 GB RAM,64 GB ROM,13MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,6000 mAh Battery,"₹7,299",https://www.flipkart.com/infinix-smart-7-night...
4,Infinix,"Infinix HOT 30i (Diamond White, 128 GB) (8 GB...",Diamond White,8 GB RAM,128 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,5000 mAh Battery,"₹8,999",https://www.flipkart.com/infinix-hot-30i-diamo...
5,Infinix,"Infinix HOT 30i (Glacier Blue, 128 GB) (8 GB ...",Glacier Blue,8 GB RAM,128 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,5000 mAh Battery,"₹8,999",https://www.flipkart.com/infinix-hot-30i-glaci...
6,Infinix,"Infinix HOT 30i (Marigold, 128 GB) (8 GB RAM)",Marigold,8 GB RAM,128 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,5000 mAh Battery,"₹8,999",https://www.flipkart.com/infinix-hot-30i-marig...
7,Infinix,"Infinix HOT 30i (Mirror Black, 128 GB) (8 GB ...",Mirror Black,8 GB RAM,128 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,5000 mAh Battery,"₹8,999",https://www.flipkart.com/infinix-hot-30i-mirro...
8,Infinix,"Infinix HOT 30i (Mirror Black, 64 GB) (4 GB RAM)",Mirror Black,4 GB RAM,64 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,5000 mAh Battery,"₹7,999",https://www.flipkart.com/infinix-hot-30i-mirro...
9,Infinix,"Infinix HOT 30i (Diamond White, 64 GB) (4 GB ...",Diamond White,4 GB RAM,64 GB ROM,50MP + AI Lens,5MP Front Camera,16.76 cm (6.6 inch) HD,5000 mAh Battery,"₹7,999",https://www.flipkart.com/infinix-hot-30i-diamo...


In [23]:
df.to_csv('smartphone.csv')

In [24]:
################## Google Maps ################

Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [25]:
maps=driver.get("https://maps.google.co.uk")

In [26]:
prompt=input("Enter a location: ")

Enter a location: Bangalore


In [27]:
inputLocation=driver.find_element(By.CLASS_NAME,'searchboxinput')
inputLocation.send_keys(prompt)

In [28]:
enterLocation=driver.find_element(By.CLASS_NAME, 'mL3xi')
enterLocation.click()

In [29]:
mapUrl = driver.current_url
print("URL Extracted: ", mapUrl)

longitude=mapUrl.split('@')[1].split(',')[0]
lattitude=mapUrl.split('@')[1].split(',')[1]
print("Longitude: ",longitude,'"N')
print("Lattitude: ",lattitude,'"E')

URL Extracted:  https://www.google.co.uk/maps/@52.4220552,-1.9628032,14z?entry=ttu
Longitude:  52.4220552 "N
Lattitude:  -1.9628032 "E


In [30]:
###################Gaming Laptop##############

Write a program to scrap all the available details of best gaming laptops from digit.in.

In [31]:
gaming=driver.get("https://www.digit.in/")

In [32]:
gamingDropdown=driver.find_element(By.CLASS_NAME, 'laptops')
gamingDropdown.click()

In [33]:
bestGamingLaptop=driver.find_element(By.XPATH, '/html/body/div[2]/div/ul/li[2]/div[2]/div/div[1]/span[4]')
bestGamingLaptop.click()

In [34]:
##I will be finding the Best Gaming Laptops Under Laptops dropdown
selectOption=driver.find_element(By.XPATH, '/html/body/div[2]/div/ul/li[2]/div[2]/div/div[5]/div/div[2]')
selectOption.click()

In [35]:
noOfProducts=driver.find_elements(By.CLASS_NAME, 'specfpro_details_cont')
print(len(noOfProducts))

7


In [36]:
lName=[]
operatingSys=[]
lDisplay=[]
lProcessor=[]
lMemory=[]
lWeight=[]
lDimension=[]
lGraph_proc=[]
lPrice=[]


#scraping name
try:    
    names=driver.find_elements(By.XPATH, "//h3")
    for n in names:
        name=n.text
        if name!='':
            lName.append(name)
        else:
            lName.append('--')
except NoSuchElementException as e:
    lName.append('No Name Available')

#scraping OS
try:
    os=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]")
    for s in os:
        sys=s.text
        if sys!='':
            operatingSys.append(sys)
        else:
            operatingSys.append('--')
except NoSuchElementException as e:
    operatingSys.append('No Name Available')

#scraping Display size
try:
    display=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[3]/td[3]")
    for d in display:
        displaySize=d.text.split(',')[0]
        if displaySize!='':
            lDisplay.append(displaySize)
        else:
            lDisplay.append('--')
except NoSuchElementException as e:
    lDisplay.append('No Name Available')

#scraping Processor
try:
    processor=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[2]/td[3]")
    for p in processor:
        pro=p.text
        if pro!='':
            lProcessor.append(pro)
        else:
            lProcessor.append('--')
except NoSuchElementException as e:
    lProcessor.append('No Name Available')
    
#scraping Memory
try:
    memory=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]")
    for p in memory:
        pro=p.text
        if pro!='':
            lMemory.append(pro)
        else:
            lMemory.append('--')
except NoSuchElementException as e:
    lMemory.append('No Name Available')
    
#scraping Weight
try:
    weight=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
    for p in weight:
        pro=p.text.split('&')[1]
        if pro!='':
            lWeight.append(pro)
        else:
            lWeight.append('--')
except NoSuchElementException as e:
    lWeight.append('No Name Available')
    
#scraping Dimension
try:
    dimension=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
    for p in dimension:
        pro=p.text.split('&')[0]
        if pro!='':
            lDimension.append(pro)
        else:
            lDimension.append('--')
except NoSuchElementException as e:
    lDimension.append('No Name Available')    

#scraping Graph_proc
try:
    graph_proc=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]")
    for p in graph_proc:
        pro=p.text
        if pro!='':
            lGraph_proc.append(pro)
        else:
            lGraph_proc.append('--')
except NoSuchElementException as e:
    lGraph_proc.append('No Name Available')    
    
#scraping Price
try:
    price=driver.find_elements(By.XPATH, "//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]")
    for p in price:
        pro=p.text
        if pro!='':
            lPrice.append(pro)
        else:
            lPrice.append('--')
except NoSuchElementException as e:
    lPrice.append('No Name Available')    

In [37]:
data={'Name':lName, 'Operating System':operatingSys, 'Display':lDisplay, 'Processor':lProcessor, 'Memory':lMemory,
                'Weight':lWeight, 'Dimension':lDimension, 'Graphics Processor':lGraph_proc, 'Price':lPrice}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
df

,Name,Operating System,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
0,HP OMEN 17 (2023),Windows 11 Home,17.3″ (2560 x 1440) screen,13th Gen Intel Core i7-13700HX 16 core process...,16 GB DDR5GB RAM & 1 TB SSD,2.78 kg weight,397.1 x 262 x 27 mm dimension,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,"₹ 269,777"
1,MSI Titan GT77 12UHS,Windows 11 Home,17.3″ (3840 x 2160) screen,12th Gen Intel Core i9-12900HX 16 core process...,64 GB DDR5GB RAM & 2 TB SSD,3.3 kg weight,397 x 330 x 23 mm dimension,16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...,"₹ 499,990"
2,Lenovo Legion 5i Pro,Windows 11 Home,16″ (2560 x 1600) screen,12th Gen Intel Core i7-12700H 14 core processo...,32 GB DDR5GB RAM & 1 TB SSD,3.6 kg weight,359.9 x 264.4 x 19.9 mm dimension,NVIDIA GeForce RTX 3070 Ti Graphics card,"₹ 211,990"
3,ASUS ROG Strix Scar 18 2023,Windows 11 Home,18″ (1920 x 1200) screen,13th Gen Intel Core i9-13980HX 24 core process...,32 GB DDR5GB RAM & 1 TB SSD,3.1 kg weight,294 x 399 x 23 mm dimension,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,"₹ 279,990"
4,Acer Predator Helios Neo 16,Windows 11 Home,16″ (2560 x 1600) screen,13th Gen Intel Core i7-13700HX processor,16 GB DDR5GB RAM & 1 TB SSD,2.6 kg weight,360 x 279 x 28 mm dimension,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,"₹ 149,990"
5,ASUS ROG Zephyrus G14,Windows 11 Home,14″ (1920 x 1200) screen,AMD Ryzen 9-6900HS 8 core processor with 4.9 G...,16 GB DDR5GB RAM & 1 TB SSD,1.65 kg weight,312 x 227 x 19 mm dimension,8 GB DDR6 AMD Radeon RX 6700S Graphics card,"₹ 156,990"
6,MSI Cyborg 15,Windows 11 Home,15.6″ (1920 x 1080) screen,12th Gen Intel Core i7-12650H 10 core processo...,16 GB DDR5GB RAM & 1 TB SSD,1.98 kg weight,--,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,"₹ 130,990"


In [38]:
################### Billionaires accross world ###########################

Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [39]:
billionaires=driver.get("https://www.forbes.com/")

In [40]:
try:    
    wait=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, "root__TFKT8")))
    x=wait.find_element(By.CLASS_NAME,'root__OblK1')
    x.click()
except TimeoutException as e:
    print("Exception raised ", e)

In [41]:
time.sleep(5)
dropdownList=driver.find_element(By.XPATH, '/html/body/div[1]/header/nav/div[1]/div[1]')
dropdownList.click()

In [42]:
time.sleep(4)
spanBtn=driver.find_element(By.XPATH, '/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]/div[1]/span')
spanBtn.click()

In [43]:
selectLink=driver.find_element(By.XPATH, '//*[@id="globalHeaderMenu"]/div/div[2]/ul/li[2]/div[2]/div[3]/ul/li[1]')
selectLink.click()

In [44]:
try:
    load=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CLASS_NAME, "TableRow_rowContainer__IC1Tv")))
except TimeoutException as e:
    print("Exception raised ", e)

In [45]:
rankNo=[]
names=[]
netWorth=[]
age=[]
citizenship=[]
source=[]
industry=[]

selectRow=driver.find_elements(By.CLASS_NAME, 'TableRow_row__L-0Km')
#print(len(selectRow))

for _ in range(15):
    driver.execute_script("window.scrollBy(0,100)")
    
#scraping first Billionar rank
try:
    highestRank=driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div')    
    hRank=highestRank.text
    if highestRank!='':
        rankNo.append(hRank)
    else:
        rankNo.append('--')
except NoSuchElementException as e:
        print("exception raised by rank: ", e)
    
#scraping rank
try:
    rank_tag = driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']/div[1]/div[1]")
    for rank in rank_tag:
        hRank=rank.text
        if hRank!='':
            rankNo.append(hRank)
        else:
            rankNo.append('--')
except NoSuchElementException as e:
        print("exception raised by rank: ", e)

#scraping first Billionar Name
try:
    firstName=driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[2]')
    fName=firstName.text
    if fName!='':
        names.append(fName) 
    else:
        names.append('--')
except NoSuchElementException as e:
        print("exception raised by name: ", e)

#scraping Name 
try:
    name = driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']/div[2]/div[1]")
    for n in name:
        fName=n.text
        if fName!='':
            names.append(fName) 
        else:
            names.append('--')
except NoSuchElementException as e:
        print("exception raised by name: ", e)
        
#scraping first Billionar Net
try:
    highestNet=driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[3]")
    hNet=highestNet.text
    if hNet!='':
        netWorth.append(hNet)
    else:
        netWorth.append('--')
except NoSuchElementException as e:
        print("exception raised by HighestNet: ", e)

#scraping Net
try:
    net = driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']/div[3]/div[1]")
    for nt in net:
        hNet=nt.text
        if hNet!='':
            netWorth.append(hNet)
        else:
            netWorth.append('--')
except NoSuchElementException as e:
        print("exception raised by HighestNet: ", e)
    
#scraping first Billionar Age
try:
    firstNameAge=driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[4]')
    bAge=firstNameAge.text
    if bAge!='':
        age.append(bAge)
    else:
        age.append('--')
except NoSuchElementException as e:
        print("exception raised by Age: ", e)

#scraping Age
try:
    bAge = driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']/div[4]/div[1]")
    for a in bAge:
        bAge=a.text
        if bAge!='':
            age.append(bAge)
        else:
            age.append('--')
except NoSuchElementException as e:
        print("exception raised by Age: ", e)
    
#scraping first Billionar Citizenship
try:
    firstNameCitizen=driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[5]')
    bCitizen=firstNameCitizen.text
    if bCitizen!='':
        citizenship.append(bCitizen)
    else:
        citizenship.append('--')
except NoSuchElementException as e:
        print("exception raised by Citizenship: ", e)
    
#scraping Citizenship
try:    
    citizen = driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']/div[5]")
    for c in citizen:
        bCitizen=c.text
        if bCitizen!='':
            citizenship.append(bCitizen)
        else:
            citizenship.append('--')
except NoSuchElementException as e:
        print("exception raised by Citizenship: ", e)

#scraping first Billionar Source
try:
    firstNameSource=driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[6]')
    sourceName=firstNameSource.text
    if sourceName!='':
        source.append(sourceName)
    else:
        source.append('--')
except NoSuchElementException as e:
        print("exception raised by Source: ", e)
    
#scraping sources
try:
    sources = driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']/div[6]/div[1]")
    for s in sources:
        sourceName=s.text
        if sourceName!='':
            source.append(sourceName)
        else:
            source.append('--')
except NoSuchElementException as e:
        print("exception raised by Source: ", e)

#scraping first Billionar Industry
try:
    firstIndustry=driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/div[7]')
    bIndustry=firstIndustry.text
    if bIndustry!='':
        industry.append(bIndustry)
    else:
        industry.append('--')
except NoSuchElementException as e:
        print("exception raised by Industry: ", e)
    
#scraping Industry
try:
    industries = driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']/div[7]/div[1]")
    for i in industries:
        bIndustry=i.text
        if bIndustry!='':
            industry.append(bIndustry)
        else:
            industry.append('--')
except NoSuchElementException as e:
        print("exception raised by Industry: ", e)

In [46]:
data={'Rank':rankNo, 'Name':names, 'NetWorth':netWorth, 'Age':age, 'Citizenship':citizenship, 'Source':source, 
      'Industry':industry}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
df

,Rank,Name,NetWorth,Age,Citizenship,Source,Industry
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,Fashion & Retail
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,$114 B,59,United States,Amazon,Technology
3,4,Larry Ellison,$107 B,78,United States,Oracle,Technology
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,195,Jin Baofang,$9.6 B,70,China,Solar panels,Energy
196,195,Luo Liguo & family,$9.6 B,67,China,Chemicals,Manufacturing
197,195,Marijke Mars,$9.6 B,58,United States,"Candy, pet food",Food & Beverage
198,195,Pamela Mars,$9.6 B,62,United States,"Candy, pet food",Food & Beverage


In [47]:
df.to_csv('billionaires.csv')

In [48]:
###############Youtube video#############################

Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video. 

In [49]:
youtube_video=driver.get("https://www.youtube.com/")

In [50]:
time.sleep(6)
wait=WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//*[@id='dialog']")))
x=wait.find_element(By.XPATH, '/html/body/ytd-app/ytd-consent-bump-v2-lightbox/tp-yt-paper-dialog/div[4]/div[2]/div[6]/div[1]/ytd-button-renderer[2]/yt-button-shape/button/yt-touch-feedback-shape/div')
x.click()

In [51]:
time.sleep(5)
try:
    search_bar = driver.find_element(By.XPATH,"//div[@class='ytd-searchbox-spt']/input")
    search_bar.send_keys("Final Battle Scene | Jurassic World") 
    #now click on search btn
    search_btn = driver.find_element(By.ID, "search-icon-legacy")
    search_btn.click()
except StaleElementReferenceException as e:
    youtube_video=driver.get("https://www.youtube.com/")
    wait=WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH,"//div[@class='ytd-searchbox-spt']/input")))
    search_bar = driver.find_element(By.XPATH,"//div[@class='ytd-searchbox-spt']/input")
    search_bar.send_keys("Final Battle Scene | Jurassic World")
    #now click on search
    search_btn = driver.find_element(By.ID, "search-icon-legacy")
    search_btn.click()

In [52]:
time.sleep(5)
#just select the first video 
select_video = driver.find_element(By.XPATH, '//*[@id="contents"]/ytd-video-renderer[1]')
select_video.click()

In [53]:
#window.scrollTo(initial_value,next_value);
time.sleep(2)
for _ in range(15):
    driver.execute_script("window.scrollBy(0,20)")
    time.sleep(2)

In [54]:
comments=[]
comment_time=[]
likesNo=[]

comment_section=driver.find_elements(By.ID, 'contents')

for c in comment_section:  
    
    #scraping Comment
    try:
        comment=driver.find_elements(By.ID, 'comment-content')
        for c in comment:
            commentText=c.text
            if commentText!='':
                comments.append(commentText)
            else:
                comments.append('--')
    except NoSuchElementException as e:
        comments.append('Comments Error', e)
        
    #scraping time of the comment
    try:
        comments_time=driver.find_elements(By.CLASS_NAME, 'published-time-text' )
        for t in comments_time:
            comment=t.text
            if comment!='':
                comment_time.append(comment)
            else:
                comment_time.append('--')
    except NoSuchElementException as e:
        comment_time.append('No Time recorded', e)

    #scraping likes for the comment
    try:
        likes=driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')
        for l in likes:
            like=l.text
            if like!='':
                likesNo.append(like)
            else:
                likesNo.append('--')
    except NoSuchElementException as e:
        likesNo.append('No Likes for video', e)

In [55]:
data={'Comment':comments, 'Time':comment_time, 'Likes':likesNo}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
df.head(500)

,Comment,Time,Likes
0,"When I saw Rexy emerge from the shadows, I imm...",11 months ago,2.1K
1,Mad respect to Claire for managing to not get ...,6 months ago (edited),3.1K
2,"Indominus: ""You can't defeat me.""\nT-Rex: ""I k...",5 months ago,1.6K
3,7:14 I love how Rexy and Blue stop attacking I...,2 weeks ago,78
4,"No matter how terrifying you are, the ocean is...",6 months ago,3K
...,...,...,...
495,The indominus rex was my favorite dinosaur ant...,3 weeks ago,14
496,I like how rexies combat style changes in roun...,6 months ago,154
497,"Experiencing this opening night, on a massive ...",5 months ago,31
498,I love how Owen whistles for them to attack. H...,1 year ago,1.8K


In [56]:
##################### Hostels in London ##############

Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall 
reviews, privates from price, dorms from price, facilities and property description.

In [130]:
try:
    driver.get("https://www.hostelworld.com/")
    load=WebDriverWait(driver, 6).until(EC.visibility_of_element_located((By.XPATH, "//div[@id='truste-consent-content']")))
    acceptBtn=driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[4]/div/div/div[2]/button[2]')
    acceptBtn.click()
except TimeoutException as e:
    print('Error caused :', e) 

In [133]:
inputLocation=driver.find_element(By.CLASS_NAME, "native-input") #"search-input-field") #
inputLocation.send_keys("London")

In [134]:
selectLondon=driver.find_element(By.XPATH, '//*[@id="3"]') #'//*[@id="predicted-search-results"]/li[2]') # "//ul[@id='predicted-search-results']//li[2]") #'//*[@id="3"]') # 
selectLondon.click()

In [135]:
searchBtn=driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/div[2]/div[2]/div/div/div/div[5]/button') #'//*[@id="__layout"]/div/div[2]/div[2]/div[2]/div/div/div/div[5]/button') #'search-button') #
searchBtn.click()

In [136]:
time.sleep(2)

In [146]:
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description = []
urls = []
    
    
# scraping  hostel name
try:
    name = driver.find_elements(By.XPATH, "//h2[@class='title title-6']") 
    for i in name:
        hName=i.text
        if hName!='':
            hostel_name.append(hName)
        else:
            hostel_name.append('--')
except NoSuchElementException as e:
    print('Hostel Name', e)


# scraping distance from city centre
try:
    dist = driver.find_elements(By.XPATH, "//div[@class='subtitle body-3']//a//span[1]") # '//*[@id="__layout"]/div/div[2]/div[4]/div/div/div[4]/div[3]/div[1]/div[1]/a/span[1]/span') # '//div[@class="subtitle body-3"]/a/span/span')# "*//span[@class='distance-description']") #
    for i in dist:
        hDistance=i.text
        if hDistance!='':
            distance.append(i.text.replace('from city centre','').replace('Hostel','').replace('-', '').replace('\n', ''))
        else:
            distance.append('--')
except NoSuchElementException as e:
    print('Hostel Distance', e)   
    
#scraping private price
try:
    pvt_price = driver.find_elements(By.XPATH, "//div[@class='prices-col']//a[@class='prices']//div[1]//div") #') #(By.XPATH, '//div[@class="property-accommodation-prices"]/div[1]/div[2]/strong') #
    for pprice in pvt_price:
        privateP=pprice.text
        if privateP!='':
            pvt_prices.append(privateP)
        else:
            pvt_prices.append('--')
except NoSuchElementException as e:
    print('Private Price', e)
    
    
#scraping dorms price
try:    
    for i in driver.find_elements(By.XPATH, "//a[@class='prices']//div[2]/div"): # "//a[@class='prices']//div[2]/div"):
        dPrice=i.text
        if dPrice!='':
            dorms_price.append(dPrice)
        else:
            dorms_price.append('--')
except NoSuchElementException as e:
    print('Dorms Price', e)
        

#scraping ratings 
try:
    hRatings=driver.find_elements(By.CLASS_NAME, 'score') #XPATH, "//div[@class='score orange big']")
    for r in hRatings:
        hRating=r.text
        if hRating!='':
            rating.append(hRating)
        else:
            rating.append('--')
except NoSuchElementException as e:
    print('Ratings', e)
    
    
#scraping reviews 
try:
    review=driver.find_elements(By.CLASS_NAME, "reviews")
    for r in review:
        hReview=r.text.strip('()').replace('Total Reviews', '')
        if hReview!='':
            reviews.append(hReview)
        else:
            reviews.append('--')
except NoSuchElementException as e:
    print('Reviews', e)

#scraping overall review
try:
    hReviews=driver.find_elements(By.CLASS_NAME, 'keyword') #, "//div[@class='keyword']//span")
    for r in hReviews:
        hReview=r.text
        if hReview!='':
            over_all.append(hReview)
        else:
            over_all.append('--')
except NoSuchElementException as e:
    print('Overall', e)


#scraping facilities
try:
    facility=driver.find_elements(By.CLASS_NAME, 'facilities')
    for f in facility:
        fac=f.text
        if fac!='':
            facilities.append(fac)
        else:
            facilities.append('--')
except NoSuchElementException as e:
    print('Facility', e)


#scraping urls
try:    
    url=driver.find_elements(By.CLASS_NAME, "nuxt-link") 
    for u in url:
        link=u.get_attribute('href') 
        urls.append(link)
except NoSuchElementException as e:
    print('URLS', e)
    

In [147]:
data={'Name':hostel_name, 'Distance from CityCenter':distance, 'Private Price':pvt_prices, 'Dorms Price':dorms_price,  
      'Rating':rating, 'Reviews':reviews, 'Over All':over_all, 'facilities':facilities}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
df

,Name,Distance from CityCenter,Private Price,Dorms Price,Rating,Reviews,Over All,facilities
0,Onefam Notting Hill by Hostel One,5.5km,£184,£29.87 £27,7.9,2059,Very Good,Free WiFi
1,St Christopher's Village,5.5km,£177,£57,9.6,12155,Superb,Free WiFi\nFollows Covid-19 sanitation guidance
2,Urbany Hostel London,1.8km,£29,£34,7.9,780,Very Good,Free WiFi\nFollows Covid-19 sanitation guidance
3,Generator London,1.8km,£66,£42.81 £35,9.4,7500,Superb,Free WiFi
4,NX London Hostel,5.4km,£277,£21,7.5,1885,Very Good,Free WiFi\nFree Breakfast
5,Safestay London Elephant & Castle,5.4km,£88,£22.62 £21,8.3,4957,Fabulous,Free WiFi\nFollows Covid-19 sanitation guidance
6,Phoenix Hostel,3km,£60 £54,£12,7.5,4126,Very Good,Free WiFi\nFollows Covid-19 sanitation guidance
7,No.8 Willesden Hostel London,3km,£150,£17,7.6,5014,Very Good,Free WiFi\nFollows Covid-19 sanitation guidance
8,Safestay London Kensington Holland Park,6.1km,£214,£13,6.5,1528,Good,Free WiFi
9,No.8 Seven Sisters,6.1km,None,£20,7.1,4010,Very Good,Free WiFi\nFollows Covid-19 sanitation guidance


In [148]:
df.to_csv("London_Hostels.csv")

In [149]:
driver.close()